# TP04

Exercice 1

In [17]:
import gymnasium as gym
import numpy as np

env = gym.make("Taxi-v3")

state_size = env.observation_space.n
action_size = env.action_space.n

policy_table = np.ones((state_size, action_size)) / action_size

value_table = np.zeros(state_size)

print("Policy table (premières lignes):")
print(policy_table[:5])
print("\nValue table (premières lignes):")
print(value_table[:5])

Policy table (premières lignes):
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]]

Value table (premières lignes):
[0. 0. 0. 0. 0.]


Exercice 2

In [14]:
num_episodes = 20

for episode in range(num_episodes):
    state, _ = env.reset()
    done = False
    total_reward = 0
    episode_actions = []
    episode_rewards = []
    
    print(f"\nÉpisode {episode + 1}:")
    while not done:
        action = np.random.choice(action_size, p=policy_table[state])
        next_state, reward, done, truncated, _ = env.step(action)
        done = done or truncated
        
        episode_actions.append(action)
        episode_rewards.append(reward)
        total_reward += reward
        state = next_state
    
    print("Actions exécutées:", episode_actions)
    print("Récompenses obtenues:", episode_rewards)
    print("Récompense totale:", total_reward)


Épisode 1:
Actions exécutées: [3, 4, 1, 4, 1, 5, 4, 3, 1, 1, 4, 2, 5, 0, 4, 3, 3, 3, 1, 5, 0, 1, 1, 4, 1, 5, 5, 4, 4, 4, 1, 3, 0, 2, 3, 4, 3, 4, 4, 4, 5, 3, 5, 4, 2, 3, 2, 3, 2, 5, 4, 4, 0, 1, 2, 4, 5, 0, 1, 2, 2, 0, 3, 3, 5, 0, 2, 0, 1, 4, 4, 1, 5, 5, 0, 5, 2, 5, 4, 1, 0, 1, 4, 2, 2, 3, 4, 3, 0, 2, 5, 3, 3, 0, 4, 1, 1, 4, 4, 0, 1, 0, 3, 5, 0, 1, 2, 4, 3, 2, 5, 3, 0, 0, 5, 2, 2, 2, 2, 1, 5, 1, 1, 5, 5, 5, 1, 3, 0, 3, 4, 3, 0, 1, 4, 2, 5, 1, 3, 4, 5, 0, 0, 3, 1, 4, 3, 0, 3, 0, 5, 5, 0, 0, 2, 3, 0, 3, 0, 4, 1, 1, 5, 3, 3, 2, 4, 0, 3, 0, 3, 4, 1, 4, 4, 2, 4, 2, 2, 0, 1, 4, 2, 5, 4, 3, 2, 1, 2, 4, 3, 4, 1, 0, 1, 3, 4, 3, 1, 2]
Récompenses obtenues: [-1, -10, -1, -10, -1, -10, -10, -1, -1, -1, -10, -1, -10, -1, -10, -1, -1, -1, -1, -10, -1, -1, -1, -10, -1, -10, -10, -10, -10, -10, -1, -1, -1, -1, -1, -10, -1, -10, -10, -10, -10, -1, -10, -10, -1, -1, -1, -1, -1, -10, -10, -10, -1, -1, -1, -10, -10, -1, -1, -1, -1, -1, -1, -1, -10, -1, -1, -1, -1, -10, -10, -1, -10, -10, -1, -10, -1, -10, 

Exercice 3

In [ ]:
gamma = 0.99
lr_policy = 0.1
clip_epsilon = 0.2

episode_states = []
episode_actions = []
episode_rewards = []

state, _ = env.reset()
done = False

while not done:
    action = np.random.choice(action_size, p=policy_table[state])
    next_state, reward, done, truncated, _ = env.step(action)
    done = done or truncated
    
    episode_states.append(state)
    episode_actions.append(action)
    episode_rewards.append(reward)
    
    state = next_state

discounted_rewards = []
running_add = 0

for r in reversed(episode_rewards):
    running_add = r + gamma * running_add
    discounted_rewards.insert(0, running_add)

discounted_rewards = np.array(discounted_rewards)
discounted_rewards = (discounted_rewards - np.mean(discounted_rewards)) / (np.std(discounted_rewards) + 1e-8)

for t in range(len(episode_states)):
    state = episode_states[t]
    action = episode_actions[t]
    
    old_prob = policy_table[state][action]
    
    advantage = discounted_rewards[t] - value_table[state]
    
    ratio = policy_table[state][action] / old_prob
    
    surr1 = ratio * advantage
    surr2 = np.clip(ratio, 1 - clip_epsilon, 1 + clip_epsilon) * advantage
    policy_loss = -np.minimum(surr1, surr2)
    
    policy_table[state][action] += lr_policy * policy_loss
    
    policy_table[state] = np.clip(policy_table[state], 1e-8, 1 - 1e-8)
    policy_table[state] /= np.sum(policy_table[state])
    
    value_table[state] += lr_policy * advantage

Mise à jour PPO terminée pour cet épisode.


Exercice 4

In [ ]:
num_eval_episodes = 20
total_rewards = []
success_count = 0

for ep in range(num_eval_episodes):
    state, _ = env.reset()
    done = False
    total_reward = 0
    steps = 0
    
    while not done and steps < 200:
        action = np.argmax(policy_table[state])
        
        next_state, reward, done, truncated, _ = env.step(action)
        done = done or truncated
        
        total_reward += reward
        state = next_state
        steps += 1
    
    total_rewards.append(total_reward)
    if reward > 0:
        success_count += 1

avg_reward = np.mean(total_rewards)
success_rate = (success_count / num_eval_episodes) * 100

print("\nRésultats après entraînement :")
print(f"- Récompense moyenne : {avg_reward:.2f}")
print(f"- Taux de réussite : {success_rate:.2f}%")

<p align="center">EL HACHYMI Ahmed Yassine</p>